# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arena コンペティションにおいて、ユーザーが選好する応答を予測するための機械学習モデルの構築に取り組んでいます。具体的には、巨大な言語モデル（LLM）であるQwen2を用いて、与えられたプロンプトに対する2つのモデルからの応答のどちらが好まれるかを予測します。

### 主要な問題
- ユーザーのプロンプトに対して、2つの匿名化された応答から、どちらがより好まれるかを明らかにするための予測モデルを構築します。

### 使用されている手法とライブラリ
1. **ライブラリのインストールとインポート**：
   - `bitsandbytes`, `transformers`, `tokenizers`, `peft` などのライブラリを用いて、最新のLLMを利用可能にします。
   - PyTorch、scikit-learn、NumPy、Pandasなど、データ処理およびモデル構築に必要なライブラリがインポートされています。

2. **データの準備**：
   - トレーニングデータとテストデータが CSV ファイルから読み込まれ、応答Aと応答Bの処理が行われます。
   - 入力テキストは、ユーザーのプロンプトとそれに対するモデルの応答を包含するように整形されます。

3. **トークナイゼーション**：
   - `AutoTokenizer`を使用して、テキストデータをトークン化し、モデルの入力として適切な形式に変換しています。

4. **モデルのロード**：
   - Qwen2モデルが初期化され、4ビット量子化を利用して効率的にGPUに配置されています。これにより、計算リソースの使用が最適化されます。
   - LoRa (Low-Rank Adaptation) によるパラメータ効率の良いファインチューニングの設定が行われています。

5. **推論**：
   - 推論には、2つのスレッドを用いてモデルの予測を効率的に行う関数が実装されています。各モデルによる出力が確率として計算され、最終的にそれぞれのモデルA、B、およびタイの結果が生成されます。

6. **出力処理**：
   - 生成された予測結果は、提出フォーマットに合う形で整形され、最終的には "submission.csv" として保存されます。

このノートブック全体を通じて、複数のLLMを活用し、複雑なデータ処理や予測を実現するための工夫が凝らされています。基本的に、二つのモデルからの応答を比較し、ユーザーの選好を予測するために、最先端の機械学習技術が利用されている点が特徴です。

---


# 用語概説 
以下に、提供されたJupyter Notebookの内容に関連して、機械学習・深層学習の初心者がつまずきそうな専門用語の解説を列挙します。

1. **bitsandbytes**:
   - 高効率なメモリ使用を実現するためのライブラリで、特に8ビットや4ビットの量子化を用いて大規模言語モデル（LLM）を効率的に動作させるのに役立ちます。このライブラリは、GPUメモリの使用を最適化するために設計されています。

2. **PEFT (Parameter Efficient Fine-Tuning)**:
   - モデル全体を訓練するのではなく、一部のパラメータ（たとえば、LoRAやAdapterなど）だけを訓練する手法で、少ないデータや計算資源で効果的にモデルの性能を向上させることができます。

3. **LoRa (Low-Rank Adaptation)**:
   - PEFTの一手法で、ニューラルネットワークの重みを低ランクで調整することで、効率的にファインチューニングを行うことができます。これにより、全体のパラメータの数を減らしながらも性能を保つことが可能です。

4. **自動混合精度 (Automatic Mixed Precision, AMP)**:
   - 実行中のモデルの計算精度を自動で調整し、計算を高速化しつつメモリ使用量を削減するための技術です。特に深層学習において、計算効率を向上させるために よく用いられます。

5. **スレッド処理 (Threading)**:
   - プログラムを同時に複数の流れで実行するための技術です。このノートブックでは、異なるGPUにモデルを分散してロードし、同時に推論を行うためにスレッドが用いられています。

6. **量子化 (Quantization)**:
   - モデルの数値を、実際に使用される数値（たとえば、浮動小数点数から整数）に変換するプロセスです。これにより、メモリ使用を削減し、計算を高速化することができます。特に、8ビットや4ビット量子化が一般的に用いられます。

7. **ログ損失 (Log Loss)**:
   - モデルの予測と実際の結果との間の誤差を評価する指標で、確率的にモデルがどれだけ的確に予測を行っているかを示します。特に2クラス分類や多クラス分類の問題において用いられることが多いです。

8. **アテンションマスク (Attention Mask)**:
   - Transformerモデルにおいて、どのトークンが計算に含まれるかを示すためのマスクです。特に、パディングされているトークンを無視するために使用されます。

9. **デバイスマップ (Device Map)**:
   - モデルがどのデバイス（CPU/GPU/TPUなど）で実行されるかを指定するためのマップで、複数のGPU環境での効率的な処理に役立ちます。

これらの解説が、ノートブックを理解するための助けとなることを期待しています。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
<div class="column-right">

# 日本語訳

```python
# bitsandbytesライブラリを最新のバージョンにインストールします。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
# transformersライブラリを最新のバージョンにインストールします。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# tokenizersライブラリを最新のバージョンにインストールします。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
# peftライブラリを最新のバージョンにインストールします。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
</details>

In [ ]:
# bitsandbytesライブラリを最新のバージョンにインストールします。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
# transformersライブラリを最新のバージョンにインストールします。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# tokenizersライブラリを最新のバージョンにインストールします。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
# peftライブラリを最新のバージョンにインストールします。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The work in this notebook is inspired by these notebooks:
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

</div>
<div class="column-right">

# 日本語訳

このノートブックでの作業は、以下のノートブックからインスピレーションを受けています：
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

## ライブラリのインポート

</div>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Importing Libraries

</div>
<div class="column-right">

# 日本語訳

# 必要なライブラリをインポートします。
import torch  # PyTorchライブラリをインポート
import sklearn  # scikit-learnライブラリをインポート
import numpy as np  # NumPyライブラリをインポート
import pandas as pd  # Pandasライブラリをインポート
import time  # 時間操作のためのライブラリをインポート

# Transformersライブラリから必要なモデルとトークナイザーをインポート
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, Qwen2ForSequenceClassification, BitsAndBytesConfig
# PEFT（Parameter Efficient Fine-Tuning）のための設定をインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast  # 自動混合精度のための機能をインポート
from threading import Thread  # スレッド処理のためのライブラリをインポート

# ガベージコレクションをインポート
import gc
import os  # OS関連の機能をインポート
import io  # 入出力操作のためのライブラリをインポート
import time  # 時間操作のためのライブラリをインポート（再度インポート、必要であれば削除可能）
import json  # JSONデータを扱うためのライブラリをインポート
import random  # ランダム操作のためのライブラリをインポート
import pickle  # Pythonオブジェクトをバイナリ形式で保存・読み込みするためのライブラリをインポート
import zipfile  # ZIPファイル操作のためのライブラリをインポート
import datetime  # 日付と時間を操作するためのライブラリをインポート
import matplotlib.pyplot as plt  # グラフ描画のためのライブラリをインポート
from IPython.display import display  # IPythonの表示機能をインポート
from collections import Counter  # 要素のカウントを行うためのCounterクラスをインポート
from collections import defaultdict  # デフォルト辞書を扱うためのdefaultdictクラスをインポート
import torch  # 再度、PyTorchライブラリをインポート
from torch import nn  # ニューラルネットワークの基盤を構築するためのライブラリをインポート
import torch.nn.functional as F  # ニューラルネットワークの機能的操作を簡単にするためのライブラリをインポート
import pytorch_lightning as pl  # PyTorch Lightningライブラリをインポート
from torch.utils.data import Dataset, DataLoader  # データセットとデータローダーのためのクラスをインポート
from sklearn.metrics import log_loss  # 対数損失を計算するための関数をインポート
import tokenizers  # トークナイザーを扱うためのライブラリをインポート

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch
import sklearn
import numpy as np
import pandas as pd
import time

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification,Qwen2ForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from threading import Thread

import gc
import os
import io
import time
import json
import random
import pickle
import zipfile
import datetime
import matplotlib.pyplot as plt
from IPython.display import display
from collections import Counter
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import tokenizers


```

</div>
<div class="column-right">

# 日本語訳

```python
# メモリ効率の良いスパース分解（SDP）を無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)
# フラッシュSDPを無効にします。
torch.backends.cuda.enable_flash_sdp(False)

# GPUが利用できない場合はメッセージを表示します。
if (not torch.cuda.is_available()): 
    print("申し訳ありませんが、GPUが必要です！")

# モデル名のパスを定義します。
MODEL_NAME = '/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1'
# 重みのパスを定義します。
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'
# 最大入力長を定義します。
MAX_LENGTH = 1284
# バッチサイズを定義します。
BATCH_SIZE = 8
# 使用するデバイスをCUDAに設定します（GPU）。
DEVICE = torch.device("cuda")
```

</div>
</details>

In [ ]:
# メモリ効率の良いスパース分解（SDP）を無効にします。
torch.backends.cuda.enable_mem_efficient_sdp(False)
# フラッシュSDPを無効にします。
torch.backends.cuda.enable_flash_sdp(False)

# GPUが利用できない場合はメッセージを表示します。
if (not torch.cuda.is_available()): 
    print("申し訳ありませんが、GPUが必要です！")

# モデル名のパスを定義します。
MODEL_NAME = '/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1'
# 重みのパスを定義します。
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'
# 最大入力長を定義します。
MAX_LENGTH = 1284
# バッチサイズを定義します。
BATCH_SIZE = 8
# 使用するデバイスをCUDAに設定します（GPU）。
DEVICE = torch.device("cuda")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("Sorry - GPU required!")

MODEL_NAME = '/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1'
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'
MAX_LENGTH = 1284
BATCH_SIZE = 8
DEVICE = torch.device("cuda")    
```

</div>
<div class="column-right">

# 日本語訳

```python
## データの準備
```

</div>
</details>

## データの準備

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Prepare Data 

</div>
<div class="column-right">

# 日本語訳

# テストデータを読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
# 訓練データを読み込みます。ファイルを読み取りモードで開きます。
train = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/train.csv', 'r'))
# サンプルの提出ファイルを読み込みます。
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
train = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/train.csv', 'r'))
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 文字列のリストを連結する関数を定義します。
def process(input_str):
    # 入力文字列の両端の'[]'を取り除きます。
    stripped_str = input_str.strip('[]')
    # 文字列を分割して、各文から余分な引用符を取り除きます。
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # 文をスペースで結合して返します。
    return  ' '.join(sentences)

# テストデータに対してプロンプト、応答A、応答Bを処理します。
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# サンプルの提出データとテストデータの先頭5行を表示します。
display(sample_sub)
display(test.head(5))

# モデル用にテキストを準備します。
test['text'] = 'ユーザープロンプト: ' + test['prompt'] +  '\n\nモデルA :\n' + test['response_a'] +'\n\n--------\n\nモデルB:\n'  + test['response_b']
# 処理されたテキストの最初の要素を表示します。
print(test['text'][0])
```

</div>
</details>

In [ ]:
# 文字列のリストを連結する関数を定義します。
def process(input_str):
    # 入力文字列の両端の'[]'を取り除きます。
    stripped_str = input_str.strip('[]')
    # 文字列を分割して、各文から余分な引用符を取り除きます。
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # 文をスペースで結合して返します。
    return  ' '.join(sentences)

# テストデータに対してプロンプト、応答A、応答Bを処理します。
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# サンプルの提出データとテストデータの先頭5行を表示します。
display(sample_sub)
display(test.head(5))

# モデル用にテキストを準備します。
test['text'] = 'ユーザープロンプト: ' + test['prompt'] +  '\n\nモデルA :\n' + test['response_a'] +'\n\n--------\n\nモデルB:\n'  + test['response_b']
# 処理されたテキストの最初の要素を表示します。
print(test['text'][0])

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# concatenate strings in list
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

display(sample_sub)
display(test.head(5))

# Prepare text for model
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + test['response_b']
print(test['text'][0])
```

</div>
<div class="column-right">

# 日本語訳

```python
## トークナイズ
```

</div>
</details>

## トークナイズ

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Tokenize

</div>
<div class="column-right">

# 日本語訳

# トークナイザーを初期化します。
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# テキストデータをトークン化します。
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

# トークン化された入力IDとアテンションマスクをデバイス（GPU）に移動させます。
INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動し、リストに変換します。
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

# 新しいデータフレームを作成し、INPUT_IDSとATTENTION_MASKSを格納します。
data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
# データフレームの最初の2行を表示します。
data[:2]

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# Move tensors to CPU and convert them to lists
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
data[:2]
```

</div>
<div class="column-right">

# 日本語訳

```python
## モデルのロード
> 各GPUに1つのモデルをロードします。
```

</div>
</details>

## モデルのロード
> 各GPUに1つのモデルをロードします。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load model 
> We load 1 model on each gpu.  

</div>
<div class="column-right">

# 日本語訳

# BitsAndBytesの設定を行います。
# 8ビットでの読み込みを設定する場合（コメントアウトされています）
# bnb_config =  BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_compute_dtype=torch.float16,
#     bnb_8bit_use_double_quant=False)

# 4ビットでの読み込みを設定します。
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビットで読み込み
    bnb_4bit_quant_type="nf4",  # 量子化のタイプを"nf4"に設定
    bnb_4bit_use_double_quant=True,  # 二重量子化を使用
    bnb_4bit_compute_dtype=torch.bfloat16,  # 計算のデータ型をbfloat16に設定
)

# GPU 0に基礎モデルをロードします。
device0 = torch.device('cuda:0')

base_model_0 = Qwen2ForSequenceClassification.from_pretrained(
    MODEL_NAME,  # モデル名を指定
    num_labels=3,  # ラベルの数を指定
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化設定を指定
    device_map='cuda:0'  # モデルをGPU 0に配置
)

# パディングトークンIDをトークナイザーから設定します。
base_model_0.config.pad_token_id = tokenizer.pad_token_id

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# BitsAndBytes configuration
# bnb_config =  BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_compute_dtype=torch.float16,
#     bnb_8bit_use_double_quant=False)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
# Load base model on GPU 0
device0 = torch.device('cuda:0')

base_model_0 = Qwen2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:0')

base_model_0.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPU 1に基礎モデルをロードします。
device1 = torch.device('cuda:1')

base_model_1 = Qwen2ForSequenceClassification.from_pretrained(
    MODEL_NAME,  # モデル名を指定
    num_labels=3,  # ラベルの数を指定
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化設定を指定
    device_map='cuda:1'  # モデルをGPU 1に配置
)

# パディングトークンIDをトークナイザーから設定します。
base_model_1.config.pad_token_id = tokenizer.pad_token_id
```

</div>
</details>

In [ ]:
# GPU 1に基礎モデルをロードします。
device1 = torch.device('cuda:1')

base_model_1 = Qwen2ForSequenceClassification.from_pretrained(
    MODEL_NAME,  # モデル名を指定
    num_labels=3,  # ラベルの数を指定
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化設定を指定
    device_map='cuda:1'  # モデルをGPU 1に配置
)

# パディングトークンIDをトークナイザーから設定します。
base_model_1.config.pad_token_id = tokenizer.pad_token_id

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load base model on GPU 1
device1 = torch.device('cuda:1')
base_model_1 = Qwen2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
base_model_1.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
## 重みのロード
```

</div>
</details>

## 重みのロード

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load weights 


</div>
<div class="column-right">

# 日本語訳

# LoRa（Low-Rank Adaptation）の設定を行います。
peft_config = LoraConfig(
    r=16,  # 決定係数レイテンシを指定
    lora_alpha=32,  # LoRaでのスケーリングファクターを指定
    lora_dropout=0.10,  # ドロップアウト率を指定
    bias='none',  # バイアスの設定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクのタイプをシーケンス分類に設定
    target_modules=['o_proj', 'v_proj']  # ターゲットモジュールを指定
)

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# LoRa configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    bias='none',
    inference_mode=True,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj'])
```

</div>
<div class="column-right">

# 日本語訳

```python
# PEFTモデルを取得し、GPU 0に配置します。
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 

# 重みをロードします（コメントアウトされています）
# model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
# モデルを評価モードに設定します。
model_0.eval()

# PEFTモデルを取得し、GPU 1に配置します。
model_1 = get_peft_model(base_model_1, peft_config).to(device1)

# 重みをロードします（コメントアウトされています）
# model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
# モデルを評価モードに設定します。
model_1.eval()

# 学習可能なパラメータを表示します。
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()
```

</div>
</details>

In [ ]:
# PEFTモデルを取得し、GPU 0に配置します。
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 

# 重みをロードします（コメントアウトされています）
# model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
# モデルを評価モードに設定します。
model_0.eval()

# PEFTモデルを取得し、GPU 1に配置します。
model_1 = get_peft_model(base_model_1, peft_config).to(device1)

# 重みをロードします（コメントアウトされています）
# model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
# モデルを評価モードに設定します。
model_1.eval()

# 学習可能なパラメータを表示します。
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Get peft
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
#Load weights
# model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_0.eval()

model_1 = get_peft_model(base_model_1, peft_config).to(device1)
# model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_1.eval()

#Trainable Parameters
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
# ガベージコレクションを実行して、不要なメモリを解放します。
gc.collect()
```

</div>
</details>

In [ ]:
# ガベージコレクションを実行して、不要なメモリを解放します。
gc.collect()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gc.collect()
```

</div>
<div class="column-right">

# 日本語訳

```python
## 推論
```

</div>
</details>

## 推論

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Inference

</div>
<div class="column-right">

# 日本語訳

# 推論を行う関数を定義します。
def inference(df, model, device, batch_size=BATCH_SIZE):
    # DataFrameから入力IDとアテンションマスクを張り込むテンソルを作成します。
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    # 各クラスの生成された確率を格納するリストを初期化します。
    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()  # モデルを評価モードに設定します。
    
    # バッチサイズごとにデータを処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを計算します。
        
        # バッチの入力IDとアテンションマスクを取得します。
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        
        with torch.no_grad():  # 勾配計算を無効にします。
            with autocast():  # 自動混合精度を使用します。
                outputs = model(
                    input_ids=batch_input_ids,  # モデルに入力IDを渡します。
                    attention_mask=batch_attention_mask  # 注意マスクを渡します。
                )
        
        # 出力のロジットをソフトマックス関数で確率に変換します。
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        # 各クラスの確率をリストに追加します。
        generated_class_a.extend(probabilities[:, 0])
        generated_class_b.extend(probabilities[:, 1])
        generated_class_c.extend(probabilities[:, 2])
    
    # DataFrameに生成されたクラスの確率を追加します。
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    # GPUのキャッシュをクリアします。
    torch.cuda.empty_cache()  

    return df  # 処理されたDataFrameを返します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        generated_class_a.extend(probabilities[:, 0])
        generated_class_b.extend(probabilities[:, 1])
        generated_class_c.extend(probabilities[:, 2])
    
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  

    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 処理時間の計測を開始します。
st = time.time()

N_SAMPLES = len(data)  # データサンプルの総数を取得します。

# データを2つのサブセットに分割します。
half = round(N_SAMPLES / 2)  # サンプル数の半分を計算します。
sub1 = data.iloc[0:half].copy()  # 最初の半分のデータを取得します。
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りの半分のデータを取得します。

# スレッド内で推論を実行するための関数を定義します。
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 推論結果を辞書に格納します。

# スレッドからの結果を保存するための辞書を初期化します。
results = {}
```

</div>
</details>

In [ ]:
# 処理時間の計測を開始します。
st = time.time()

N_SAMPLES = len(data)  # データサンプルの総数を取得します。

# データを2つのサブセットに分割します。
half = round(N_SAMPLES / 2)  # サンプル数の半分を計算します。
sub1 = data.iloc[0:half].copy()  # 最初の半分のデータを取得します。
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りの半分のデータを取得します。

# スレッド内で推論を実行するための関数を定義します。
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 推論結果を辞書に格納します。

# スレッドからの結果を保存するための辞書を初期化します。
results = {}

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

N_SAMPLES = len(data)

# Split the data into two subsets
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()
sub2 = data.iloc[half:N_SAMPLES].copy()

# Function to run inference in a thread
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)

# Dictionary to store results from threads
results = {}
```

</div>
<div class="column-right">

# 日本語訳

```python
# スレッドを開始します。
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # スレッドt0を作成
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # スレッドt1を作成

t0.start()  # スレッドt0を開始
t1.start()  # スレッドt1を開始

# すべてのスレッドの終了を待ちます。
t0.join()
t1.join()

# 結果を元のDataFrameに結合します。
data = pd.concat([results[0], results[1]], axis=0)

# 処理が完了したことを表示します。
print(f"処理が完了しました。合計時間: {time.time() - st}")

# ターゲット列を定義します。
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

# サンプルの提出データフレームに結果を追加します。
sample_sub[TARGETS] = data[TARGETS]
```

</div>
</details>

In [ ]:
# スレッドを開始します。
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # スレッドt0を作成
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # スレッドt1を作成

t0.start()  # スレッドt0を開始
t1.start()  # スレッドt1を開始

# すべてのスレッドの終了を待ちます。
t0.join()
t1.join()

# 結果を元のDataFrameに結合します。
data = pd.concat([results[0], results[1]], axis=0)

# 処理が完了したことを表示します。
print(f"処理が完了しました。合計時間: {time.time() - st}")

# ターゲット列を定義します。
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

# サンプルの提出データフレームに結果を追加します。
sample_sub[TARGETS] = data[TARGETS]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# start threads
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))

t0.start()
t1.start()

# Wait for all threads to finish
t0.join()
t1.join()

# Combine results back into the original DataFrame
data = pd.concat([results[0], results[1]], axis=0)

print(f"Processing complete. Total time: {time.time() - st}")

TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = data[TARGETS]
```

</div>
<div class="column-right">

# 日本語訳

```python
# ターゲット列の値を取得し、予測結果を変数に格納します。
llama_preds = data[TARGETS].values
```

</div>
</details>

In [ ]:
# ターゲット列の値を取得し、予測結果を変数に格納します。
llama_preds = data[TARGETS].values

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
llama_preds = data[TARGETS].values
```

</div>
<div class="column-right">

# 日本語訳

```python
# 予測結果をDataFrameに変換し、元のテストデータのIDをインデックスとして設定します。
out = pd.DataFrame(llama_preds, 
                index=test.id,  # テストデータのIDをインデックスに設定
                columns=train.columns[-3:])  # 直近の3つの列をカラムとして設定
# 結果の先頭5行を表示します。
display(out.head())
```

</div>
</details>

In [ ]:
# 予測結果をDataFrameに変換し、元のテストデータのIDをインデックスとして設定します。
out = pd.DataFrame(llama_preds, 
                index=test.id,  # テストデータのIDをインデックスに設定
                columns=train.columns[-3:])  # 直近の3つの列をカラムとして設定
# 結果の先頭5行を表示します。
display(out.head())

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

out = pd.DataFrame(llama_preds, 
                index = test.id, 
                    columns = train.columns[-3:])
display(out.head())

```

</div>
<div class="column-right">

# 日本語訳

```python
# 予測結果をCSVファイルとして保存します。
out.to_csv('submission.csv')
```

</div>
</details>

In [ ]:
# 予測結果をCSVファイルとして保存します。
out.to_csv('submission.csv')